In [2]:
from src.mtal.data_collect import get_spot_pairs

cryptos = get_spot_pairs()

In [35]:
from src.mtal.data_collect import get_pair_df

# df = get_pair_df(pair="BTCUSDT")
df = get_pair_df(pair="BTCUSDT", limit=1000, frequency="1w")
df.head()

,Open Time,Open,High,Low,Close,Volume,Close Time
0,2017-08-14,4261.48,4485.39,3850.00,4086.29,2843.431426,2017-08-20 23:59:59.999
1,2017-08-21,4069.13,4453.91,3400.00,4310.01,4599.396629,2017-08-27 23:59:59.999
2,2017-08-28,4310.01,4939.19,4124.54,4509.08,4753.843376,2017-09-03 23:59:59.999
3,2017-09-04,4505.00,4788.59,3603.00,4130.37,6382.787745,2017-09-10 23:59:59.999
4,2017-09-11,4153.62,4394.59,2817.00,3699.99,8106.705127,2017-09-17 23:59:59.999


In [33]:
from src.mtal.utils import generate_pinescript
from src.mtal.backtesting.ema_cross_backtest import EMACrossBacktester 

short_ema = 25
long_ema = 50
tester = EMACrossBacktester(
    short_ema=short_ema, long_ema=long_ema, data=df
)
results = tester.run()

entry_dates = [int(entry_date.timestamp() * 1000) for entry_date in results.entry_dates]
exit_dates = [int(exit_date.timestamp() * 1000) for exit_date in results.exit_dates]

print(results)
#print(generate_pinescript(entry_dates, exit_dates))


BacktestResults(pnl=9655.398464334387, pnl_percentage=8.655398464334386, max_drawdown=-0.15080939487379386, win_rate=0.6, average_return=0.862827167733227, trade_number=5, entry_dates=[Timestamp('2017-08-27 23:59:59.999000'), Timestamp('2017-10-01 23:59:59.999000'), Timestamp('2019-06-09 23:59:59.999000'), Timestamp('2020-05-10 23:59:59.999000'), Timestamp('2023-07-02 23:59:59.999000')], exit_dates=[Timestamp('2017-09-24 23:59:59.999000'), Timestamp('2018-08-26 23:59:59.999000'), Timestamp('2020-04-05 23:59:59.999000'), Timestamp('2022-05-08 23:59:59.999000'), Timestamp('2024-04-14 23:59:59.999000')])


In [28]:
list([int(i) for i in df['Close Time'].astype(int) / 10**6])

[1503273599999,
 1503878399999,
 1504483199999,
 1505087999999,
 1505692799999,
 1506297599999,
 1506902399999,
 1507507199999,
 1508111999999,
 1508716799999,
 1509321599999,
 1509926399999,
 1510531199999,
 1511135999999,
 1511740799999,
 1512345599999,
 1512950399999,
 1513555199999,
 1514159999999,
 1514764799999,
 1515369599999,
 1515974399999,
 1516579199999,
 1517183999999,
 1517788799999,
 1518393599999,
 1518998399999,
 1519603199999,
 1520207999999,
 1520812799999,
 1521417599999,
 1522022399999,
 1522627199999,
 1523231999999,
 1523836799999,
 1524441599999,
 1525046399999,
 1525651199999,
 1526255999999,
 1526860799999,
 1527465599999,
 1528070399999,
 1528675199999,
 1529279999999,
 1529884799999,
 1530489599999,
 1531094399999,
 1531699199999,
 1532303999999,
 1532908799999,
 1533513599999,
 1534118399999,
 1534723199999,
 1535327999999,
 1535932799999,
 1536537599999,
 1537142399999,
 1537747199999,
 1538351999999,
 1538956799999,
 1539561599999,
 1540166399999,
 1540771

In [36]:
from itertools import combinations
from src.mtal.backtesting.ema_cross_backtest import EMACrossBacktester

# Charger les données

# Plage des valeurs pour EMAs
ema_values = range(1, 60)  # Exemple de 1 à 49

# Générer des combinaisons où short_ema < long_ema
combinations = [(short_ema, long_ema) for short_ema, long_ema in combinations(ema_values, 2)]

# Dictionnaire pour stocker les résultats
results = {}

# Tester toutes les combinaisons
for short_ema, long_ema in combinations:
    tester = EMACrossBacktester(short_ema=short_ema, long_ema=long_ema, data=df)
    test_results = tester.run()
    results[(short_ema, long_ema)] = test_results

# Trouver la meilleure combinaison basée sur un critère spécifique
best_combination = max(results, key=lambda x: results[x].pnl)  # Ajuster le critère si nécessaire
best_result = results[best_combination]

print(f"Meilleure combinaison: Short EMA = {best_combination[0]}, Long EMA = {best_combination[1]}")
print(f"Résultat du test: {best_result}")


Meilleure combinaison: Short EMA = 2, Long EMA = 4
Résultat du test: BacktestResults(pnl=47859.256257907065, pnl_percentage=46.85925625790706, max_drawdown=-0.17933953725294596, win_rate=0.41379310344827586, average_return=0.26790017556481827, trade_number=29, entry_dates=[Timestamp('2017-08-27 23:59:59.999000'), Timestamp('2017-10-01 23:59:59.999000'), Timestamp('2018-03-04 23:59:59.999000'), Timestamp('2018-04-15 23:59:59.999000'), Timestamp('2018-07-22 23:59:59.999000'), Timestamp('2018-09-02 23:59:59.999000'), Timestamp('2019-01-06 23:59:59.999000'), Timestamp('2019-02-10 23:59:59.999000'), Timestamp('2019-08-04 23:59:59.999000'), Timestamp('2019-09-15 23:59:59.999000'), Timestamp('2019-10-27 23:59:59.999000'), Timestamp('2020-01-12 23:59:59.999000'), Timestamp('2020-04-12 23:59:59.999000'), Timestamp('2020-07-12 23:59:59.999000'), Timestamp('2020-10-11 23:59:59.999000'), Timestamp('2021-05-02 23:59:59.999000'), Timestamp('2021-07-25 23:59:59.999000'), Timestamp('2021-10-03 23:59:5

In [30]:
entry_dates = [int(entry_date.timestamp() * 1000) for entry_date in best_result.entry_dates]
exit_dates = [int(exit_date.timestamp() * 1000) for exit_date in best_result.exit_dates]

print(generate_pinescript(entry_dates, exit_dates))


        //@version=5
        indicator("Manual Trades", overlay=true)

        var int[] entryDates = array.new_int()
        var int[] exitDates = array.new_int()

     
        array.push(entryDates, 1503878399999)
        array.push(entryDates, 1506902399999)
        array.push(entryDates, 1520207999999)
        array.push(entryDates, 1523836799999)
        array.push(entryDates, 1532303999999)
        array.push(entryDates, 1535932799999)
        array.push(entryDates, 1546819199999)
        array.push(entryDates, 1549843199999)
        array.push(entryDates, 1564963199999)
        array.push(entryDates, 1568591999999)
        array.push(entryDates, 1572220799999)
        array.push(entryDates, 1578873599999)
        array.push(entryDates, 1586735999999)
        array.push(entryDates, 1594598399999)
        array.push(entryDates, 1602460799999)
        array.push(entryDates, 1619999999999)
        array.push(entryDates, 1627257599999)
        array.push(entryDates, 1633305599999)
